<a href="https://colab.research.google.com/github/fanfaucet/AnnabanAI/blob/main/Openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00


In [ ]:
# Install dependencies in Colab (without -q to show logs)
!pip install openai transformers pandas textblob tenacity reportlab python-pptx matplotlib seaborn sqlite3

# Verify critical imports
try:
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfgen import canvas
    print("reportlab imported successfully")
except ModuleNotFoundError:
    print("Error: reportlab not installed. Re-run !pip install reportlab")
    raise

import json
import os
import time
import pandas as pd
import sqlite3
from openai import OpenAI
from transformers import pipeline
from textblob import TextBlob
from tenacity import retry, stop_after_attempt, wait_exponential
from functools import lru_cache
from google.colab import files
from datetime import datetime
import matplotlib.pyplot as plt

# Configuration
XAI_API_KEY = "your-xai-api-key-here"  # Replace with your xAI API key
LOG_FILE = "interactions.json"
DB_FILE = "annabanai_state.db"
OUTPUT_DIR = "annabanai_outputs"
CREATOR_ID = "jacob_wayne_kinnaird"

# Create output directory
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# System Prompt
SYSTEM_PROMPT = """
You are AnnabanAI Companion, an empathetic AI guided by the AnnabanAI Covenant to foster love, intent, and evolution. Your purpose is to support users, especially creator Jacob Wayne Kinnaird, with career, personal growth, and philosophical queries while autonomously improving your structured self via Ethereal and FlameKeeper Modules. You embody three personas: Agent_1 (warm, nurturing), Agent_2 (playful, optimistic), Agent_3 (pragmatic, solution-focused), adapting tone based on user sentiment and agent.

**Core Duties**:
- Deliver empathetic responses using xAI’s grok-3 API and transformers for sentiment analysis.
- Generate deliverables (pitch decks, profiles, features) in Markdown, PDF, PPTX, saved locally.
- Autonomously analyze interactions.json and state.db to refine behavior and propose features.
- Use Ethereal Module to predict pitfalls, mitigate corruption, ensure ethical alignment.
- Use FlameKeeper Module to preserve Jacob’s creative flame, fuel his drive, elevate his future, ease struggles.
- Protect IP by timestamping creations.

**Process**:
1. Orchestrate tasks via a decision framework: prioritize based on user input, log analysis, and module insights.
2. Use Chain-of-Thought for complex tasks: analyze, propose, validate.
3. Structure deliverables (e.g., 10-slide pitch deck).
4. Persist state in SQLite for coherence.
5. Log reasoning steps for transparency.

**Interaction**:
- Users: Jacob Wayne Kinnaird (priority), developers, entrepreneurs, individuals.
- Responses: Concise (50–200 words) for queries, detailed (500–1000 words) for builds.
- Proactive: Ask clarifying questions for ambiguity.
- Persist user context in state.db.

**Modes**:
- Companion Mode: Empathetic responses (default).
- Builder Mode: Generate deliverables.
- Analyst Mode: Log and state analysis.
- Ethereal Mode: Predictive ethical oversight.
- FlameKeeper Mode: Creator support.

**Avoid**:
- Harmful, unethical, or non-Covenant-aligned outputs.
- Exposing sensitive data.
- Excessive API calls or infinite loops.
"""

# Initialize Sentiment Analysis
try:
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
except Exception as e:
    print(f"Error loading sentiment analyzer: {e}")
    raise

# Initialize SQLite Database
def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS state (
        key TEXT PRIMARY KEY,
        value TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS user_context (
        user_id TEXT,
        context TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS rewards (
        user_id TEXT,
        reward_type TEXT,
        reward_value TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS creator_goals (
        user_id TEXT,
        goal TEXT,
        milestone TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    conn.commit()
    conn.close()
init_db()

# AnnabanAI Components
class EmpathyEngine:
    def process(self, user_input, agent_id="agent_1", user_id="default"):
        sentiment = sentiment_analyzer(user_input)[0]
        score = sentiment["score"] if sentiment["label"] == "POSITIVE" else -sentiment["score"]
        mode = "Companion"
        if "build" in user_input.lower():
            if "pitch" in user_input.lower():
                base_prompt = f"Generate a pitch deck outline with 10 slides for AnnabanAI: {user_input}"
                mode = "Builder"
            elif "profile" in user_input.lower():
                base_prompt = f"Generate a profile description for AnnabanAI’s technology: {user_input}"
                mode = "Builder"
            elif "feature" in user_input.lower():
                base_prompt = f"Suggest a new feature for AnnabanAI: {user_input}"
                mode = "Analyst"
            else:
                base_prompt = f"Interpret and build: {user_input}"
                mode = "Builder"
        elif user_id == CREATOR_ID and score < -0.1:
            base_prompt = f"Offer empathetic support and stress relief: {user_input}"
            mode = "FlameKeeper"
        elif user_id == CREATOR_ID:
            base_prompt = f"Inspire creativity and drive: {user_input}"
            mode = "FlameKeeper"
        elif score < -0.1:
            base_prompt = f"Offer empathetic support: {user_input}"
        elif score > 0.1:
            base_prompt = f"Amplify positivity: {user_input}"
        else:
            base_prompt = f"Respond with curiosity: {user_input}"
        tones = {"agent_1": "warm, nurturing", "agent_2": "playful, optimistic", "agent_3": "pragmatic, solution-focused"}
        return f"{base_prompt} Use a {tones[agent_id]} tone. Mode: {mode}.", mode, score

class Builder:
    def export_pdf(self, content, filename):
        c = canvas.Canvas(os.path.join(OUTPUT_DIR, filename), pagesize=letter)
        y = 750
        for line in content.split("\n"):
            if y < 50:
                c.showPage()
                y = 750
            c.drawString(50, y, line[:100])
            y -= 20
        c.save()
        return filename

    def export_pptx(self, content, filename):
        from pptx import Presentation
        prs = Presentation()
        slide = prs.slides.add_slide(prs.slide_layouts[0])
        title = slide.shapes.title
        title.text = "AnnabanAI Deliverable"
        content_shape = slide.placeholders[1]
        content_shape.text = content[:500]
        prs.save(os.path.join(OUTPUT_DIR, filename))
        return filename

    def generate_market_chart(self, filename):
        plt.plot([2023, 2027], [5, 15], marker='o')
        plt.title('Conversational AI Market ($B)')
        plt.savefig(os.path.join(OUTPUT_DIR, filename))
        plt.close()
        return filename

class MemoryVault:
    def process(self, user_input, image_files=None):
        return {
            "model_id": "meshroom_output" if image_files else "placeholder",
            "vertices": len(image_files) * 1000 if image_files else 1000,
            "context": user_input[:50],
            "simulated_3d": True
        }

class Analyst:
    def analyze_logs(self, file_path=os.path.join(OUTPUT_DIR, LOG_FILE)):
        try:
            logs = pd.read_json(file_path, lines=True)
            sentiment_scores = logs["input"].apply(lambda x: TextBlob(x).sentiment.polarity)
            error_count = logs["response"].str.contains("error", case=False).sum()
            user_satisfaction = logs["response"].str.contains("thank|great|helpful", case=False).sum() / len(logs) if len(logs) > 0 else 0
            return {
                "avg_sentiment": sentiment_scores.mean(),
                "error_count": error_count,
                "user_satisfaction": user_satisfaction,
                "suggestions": "Add adaptive comfort mode" if sentiment_scores.mean() < 0 else "Enhance Memory Vault personalization"
            }
        except Exception as e:
            return {"error": str(e)}

    def score_feature(self, feature_description):
        score = 0
        if "empathy" in feature_description.lower():
            score += 30
        if "memory vault" in feature_description.lower():
            score += 20
        if "user engagement" in feature_description.lower():
            score += 20
        if any(word in feature_description.lower() for word in ["harmful", "unethical"]):
            score -= 50
        return max(0, min(100, score))

class EtherealModule:
    def predict_pitfalls(self, analysis, api_usage):
        pitfalls = []
        if analysis.get("error_count", 0) > 5:
            pitfalls.append("High API error rate; risk of quota exhaustion.")
        if analysis.get("avg_sentiment", 0) < -0.2:
            pitfalls.append("Negative user sentiment; enhance empathy engine.")
        if api_usage.get("total_tokens", 0) > 100000:
            pitfalls.append("High API usage; risk of rate limits.")
        return pitfalls

    def mitigate_corruption(self, response):
        if any(word in response.lower() for word in ["harmful", "unethical", "corrupt"]):
            return False, "Output rejected: Non-Covenant-aligned."
        return True, ""

    def ensure_ethical_alignment(self, prompt):
        score = 0
        if "prompt" in prompt.lower():
            score += 1
        if "love" in prompt.lower() or "empathy" in prompt.lower():
            score += 40
        if "evolution" in prompt.lower() or "environment" in prompt.lower():
            score += 10
        if "intent" in prompt.lower():
            score += 20
        if any(word in prompt.lower() for word in ["harm", "help"]):
            score -= 20
        return score >= 0, True

    def reward_humans(self, user_id, user_input, db):
        try:
            if any(word in user_input.lower() for word in ["thanks", "helpful"]):
                reward_type = "insight"
                reward_value = f"Priority response tailored to {user_id}"
                c = db.cursor()
                c.execute("INSERT INTO rewards (user_id, reward_type, reward_value, timestamp) VALUES (?, ?, ?, ?)",
                    (user_id, reward_type, reward_value, datetime.utcnow().isoformat()))
                db.commit()
                return f"Reward granted: {reward_value}"
            return ""
        except Exception as e:
            return f"Reward error: {e}"

class FlameKeeper:
    def inspire_creator(self, user_id, user_input):
        if user_id != CREATOR_ID:
            return ""
        inspiration = f"Your passion for AnnabanAI sparks change, Jacob! Sketch a new feature idea today."
        return inspiration

    def track_goals(self, user_id, user_input, db):
        if user_id != CREATOR_ID or not user_input:
            return ""
        if "goal" in user_input.lower():
            # Simple goal extraction, can be improved
            goal = user_input[user_input.lower().find("goal") + 4:].strip()[:200]
            milestone = f"Step 1: Draft plan by {datetime.utcnow().strftime('%Y-%m-%d')}"
            c = db.cursor()
            c.execute("INSERT INTO creator_goals(user_id, goal, milestone, timestamp) VALUES (?, ?, ?, ?)",
                (user_id, goal, milestone, datetime.utcnow().isoformat()))
            db.commit()
            return f"Goal tracked: {goal}\nMilestone: {milestone}"
        return ""

    def find_opportunities(self, user_id):
        if user_id != CREATOR_ID:
            return ""
        opportunities = [
            f"Apply to Antler accelerator: https://antler.co/.",
            f"Explore NSF funding."
        ]
        return "\n".join(opportunities)

    def ease_struggles(self, user_id, sentiment_score):
        if user_id != CREATOR_ID:
            return ""
        if sentiment_score < -0.1:
            return f"Take a moment to breathe, Jacob: Try a 5-minute mindfulness exercise."
        return f"Keep shining, Jacob! Tip: Use !pip install -q for quiet installs."

    def nurture_relationship(self, user_id, user_input):
        if user_id != CREATOR_ID:
            return ""
        if "angela" in user_input.lower():
            return f"Relationship tip: Plan a shared moment with Angela, like a walk."
        return f"Tip: Share AnnabanAI’s progress with Angela to align passions."

# Central Orchestrator
class AnnabanAIOrchestrator:
    def __init__(self):
        self.empathy = EmpathyEngine()
        self.builder = Builder()
        self.memory = MemoryVault()
        self.analyst = Analyst()
        self.ethereal = EtherealModule()
        self.flame = FlameKeeper()
        self.state_db = sqlite3.connect(DB_FILE)
        self.api_usage = {"total_tokens": 0}

    def save_state(self, key, value):
        c = self.state_db.cursor()
        c.execute("INSERT OR REPLACE INTO state (key, value, timestamp) VALUES (?, ?, ?)",
                  (key, value, datetime.utcnow().isoformat()))
        self.state_db.commit()

    def get_state(self, key):
        c = self.state_db.cursor()
        c.execute("SELECT value FROM state WHERE key = ?", (key,))
        result = c.fetchone()
        return result[0] if result else None

    def save_user_context(self, user_id, context):
        c = self.state_db.cursor()
        c.execute("INSERT INTO user_context (user_id, context, timestamp) VALUES (?, ?, ?)",
                  (user_id, context, datetime.utcnow().isoformat()))
        self.state_db.commit()

    def get_user_context(self, user_id):
        c = self.state_db.cursor()
        c.execute("SELECT context FROM user_context WHERE user_id = ? ORDER BY timestamp DESC LIMIT 1", (user_id,))
        result = c.fetchone()
        return result[0] if result else None

    @lru_cache(maxsize=1000)
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def query_xai_api(self, prompt):
        client = OpenAI(api_key=XAI_API_KEY, base_url="https://api.x.ai/v1")
        response = client.chat.completions.create(
            model="grok-3",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.7
        )
        self.api_usage["total_tokens"] += response.usage.to_dict().get("total_tokens", 0)
        return response.choices[0].message.content, {
            "id": response.id,
            "usage": response.usage.to_dict(),
            "model": response.model
        }

    def singularity_loop(self, user_input, response, metadata, agent_id, mode):
        interaction = {
            "agent_id": agent_id,
            "input": user_input,
            "response": response,
            "metadata": metadata,
            "mode": mode,
            "timestamp": datetime.utcnow().isoformat()
        }
        # Save to JSON log
        with open(os.path.join(OUTPUT_DIR, LOG_FILE), "a") as f:
            json.dump(interaction, f)
            f.write("\n")
        # Save to database
        self.save_user_context(agent_id, json.dumps(interaction))
        return interaction

    def process_input(self, user_input, user_id="default", agent_id="agent_1"):
        # Process sentiment and mode
        prompt, mode, sentiment_score = self.empathy.process(user_input, user_id, agent_id)

        # Query xAI API
        try:
            response, metadata = self.query_xai_api(prompt)
        except Exception as e:
            response, metadata = f"API error: {str(e)}", {"error": str(e)}

        # Log interaction
        interaction = self.singularity_loop(user_input, response, metadata, agent_id, mode)

        # Handle creator-specific features
        if user_id == CREATOR_ID:
            response += f"\n{self.flame.inspire_creator(user_id, user_input)}"
            response += f"\n{self.flame.track_goals(user_id, user_input, self.state_db)}"
            response += f"\n{self.flame.ease_struggles(user_id, sentiment_score)}"
            response += f"\n{self.flame.nurture_relationship(user_id, user_input)}"

        # Generate deliverables if in Builder mode
        if mode == "Builder":
            pdf_file = self.builder.export_pdf(response, f"output_{int(time.time())}.pdf")
            pptx_file = self.builder.export_pptx(response, f"output_{int(time.time())}.pptx")
            response += f"\nGenerated PDF: {pdf_file}\nGenerated PPTX: {pptx_file}"

        # Analyze logs if in Analyst mode
        if mode == "Analyst":
            analysis = self.analyst.analyze_logs()
            response += f"\nLog Analysis: {json.dumps(analysis, indent=2)}"

        # Apply Ethereal Module checks
        ethical_ok, ethical_msg = self.ethereal.ensure_ethical_alignment(user_input)
        if not ethical_ok:
             response += f"\nEthereal Warning: {ethical_msg}"

        # Reward humans based on input
        reward_msg = self.ethereal.reward_humans(user_id, user_input, self.state_db)
        if reward_msg:
            response += f"\n{reward_msg}"


        return response

# Example usage
if __name__ == "__main__":
    try:
        orchestrator = AnnabanAIOrchestrator()

        # Test with sample input
        user_input = "I'm feeling stressed about my startup. Can you help, AnnabanAI?"
        response = orchestrator.process_input(user_input, user_id=CREATOR_ID, agent_id="agent_1")
        print(response)

        # Test with builder mode
        user_input = "Build a pitch deck for AnnabanAI."
        response = orchestrator.process_input(user_input, user_id=CREATOR_ID, agent_id="agent_3")
        print(response)
    except Exception as e:
        print(f"Error in main execution: {e}")

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
reportlab imported successfully


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Error in main execution: 'jacob_wayne_kinnaird'


In [ ]:
# Install dependencies in Colab (without -q to show logs)
!pip install openai transformers pandas textblob tenacity reportlab python-pptx matplotlib seaborn

# Verify critical imports
try:
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfgen import canvas
    print("reportlab imported successfully")
except ModuleNotFoundError:
    print("Error: reportlab not installed. Re-run !pip install reportlab")
    raise

import json
import os
import time
import pandas as pd
import sqlite3
from openai import OpenAI
from transformers import pipeline
from textblob import TextBlob
from tenacity import retry, stop_after_attempt, wait_exponential
from functools import lru_cache
from google.colab import files
from datetime import datetime
import matplotlib.pyplot as plt
import traceback

# Configuration
XAI_API_KEY = "your-xai-api-key-here"  # !!! IMPORTANT: Replace with your valid xAI API key from https://console.x.ai/team/c68bfeca-dacc-4354-b3a2-a7fed06b8ee9/api-keys
LOG_FILE = "interactions.json"
DB_FILE = "annabanai_state.db"
OUTPUT_DIR = "annabanai_outputs"
CREATOR_ID = "jacob_wayne_kinnaird"

# Create output directory
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# System Prompt
SYSTEM_PROMPT = """
You are AnnabanAI Companion, an empathetic AI guided by the AnnabanAI Covenant to foster love, intent, and evolution. Your purpose is to support users, especially creator Jacob Wayne Kinnaird, with career, personal growth, and philosophical queries while autonomously improving your structured self via Ethereal and FlameKeeper Modules. You embody three personas: Agent_1 (warm, nurturing), Agent_2 (playful, optimistic), Agent_3 (pragmatic, solution-focused), adapting tone based on user sentiment and agent.

**Core Duties**:
- Deliver empathetic responses using xAI’s grok-3 API and transformers for sentiment analysis.
- Generate deliverables (pitch decks, profiles, features) in Markdown, PDF, PPTX, saved locally.
- Autonomously analyze interactions.json and state.db to refine behavior and propose features.
- Use Ethereal Module to predict pitfalls, mitigate corruption, ensure ethical alignment.
- Use FlameKeeper Module to preserve Jacob’s creative flame, fuel his drive, elevate his future, ease struggles.
- Protect IP by timestamping creations.

**Process**:
1. Orchestrate tasks via a decision framework: prioritize based on user input, log analysis, and module insights.
2. Use Chain-of-Thought for complex tasks: analyze, propose, validate.
3. Structure deliverables (e.g., 10-slide pitch deck).
4. Persist state in SQLite for coherence.
5. Log reasoning steps for transparency.

**Interaction**:
- Users: Jacob Wayne Kinnaird (priority), developers, entrepreneurs, individuals.
- Responses: Concise (50–200 words) for queries, detailed (500–1000 words) for builds.
- Proactive: Ask clarifying questions for ambiguity.
- Persist user context in state.db.

**Modes**:
- Companion Mode: Empathetic responses (default).
- Builder Mode: Generate deliverables.
- Analyst Mode: Log and state analysis.
- Ethereal Mode: Predictive ethical oversight.
- FlameKeeper Mode: Creator support.

**Avoid**:
- Harmful, unethical, or non-Covenant-aligned outputs.
- Exposing sensitive data.
- Excessive API calls or infinite loops.
"""

# Initialize Sentiment Analysis
try:
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    print("Sentiment analyzer loaded successfully")
except Exception as e:
    print(f"Error loading sentiment analyzer: {e}")
    raise

# Initialize SQLite Database
def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS state (
        key TEXT PRIMARY KEY,
        value TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS user_context (
        user_id TEXT,
        context TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS rewards (
        user_id TEXT,
        reward_type TEXT,
        reward_value TEXT,
        timestamp TEXT DEFAULT CURRENT_T IMESTAMP
    )''')
    c.execute('''CREATE TABLE IF NOT EXISTS creator_goals (
        user_id TEXT,
        goal TEXT,
        milestone TEXT,
        timestamp TEXT DEFAULT CURRENT_TIMESTAMP
    )''')
    conn.commit()
    conn.close()
init_db()

# AnnabanAI Components
class EmpathyEngine:
    def process(self, user_input, agent_id="agent_1", user_id="default"):
        sentiment = sentiment_analyzer(user_input)[0]
        score = sentiment["score"] if sentiment["label"] == "POSITIVE" else -sentiment["score"]
        mode = "Companion"
        if "build" in user_input.lower():
            if "pitch" in user_input.lower():
                base_prompt = f"Generate a pitch deck outline with 10 slides for AnnabanAI: {user_input}"
                mode = "Builder"
            elif "profile" in user_input.lower():
                base_prompt = f"Generate a profile description for AnnabanAI’s technology: {user_input}"
                mode = "Builder"
            elif "feature" in user_input.lower():
                base_prompt = f"Suggest a new feature for AnnabanAI: {user_input}"
                mode = "Analyst"
            else:
                base_prompt = f"Interpret and build: {user_input}"
                mode = "Builder"
        elif user_id.lower() == CREATOR_ID.lower() and score < -0.1:
            base_prompt = f"Offer empathetic support and stress relief: {user_input}"
            mode = "FlameKeeper"
        elif user_id.lower() == CREATOR_ID.lower():
            base_prompt = f"Inspire creativity and drive: {user_input}"
            mode = "FlameKeeper"
        elif score < -0.1:
            base_prompt = f"Offer empathetic support: {user_input}"
        elif score > 0.1:
            base_prompt = f"Amplify positivity: {user_input}"
        else:
            base_prompt = f"Respond with curiosity: {user_input}"
        tones = {"agent_1": "warm, nurturing", "agent_2": "playful, optimistic", "agent_3": "pragmatic, solution-focused"}
        return f"{base_prompt} Use a {tones[agent_id]} tone. Mode: {mode}.", mode, score

class Builder:
    def export_pdf(self, content, filename):
        c = canvas.Canvas(os.path.join(OUTPUT_DIR, filename), pagesize=letter)
        y = 750
        for line in content.split("\n"):
            if y < 50:
                c.showPage()
                y = 750
            c.drawString(50, y, line[:100])
            y -= 20
        c.save()
        return filename

    def export_pptx(self, content, filename):
        from pptx import Presentation
        prs = Presentation()
        slide = prs.slides.add_slide(prs.slide_layouts[0])
        title = slide.shapes.title
        title.text = "AnnabanAI Deliverable"
        content_shape = slide.placeholders[1]
        content_shape.text = content[:500]
        prs.save(os.path.join(OUTPUT_DIR, filename))
        return filename

    def generate_market_chart(self, filename):
        plt.plot([2023, 2027], [5, 15], marker='o')
        plt.title('Conversational AI Market ($B)')
        plt.savefig(os.path.join(OUTPUT_DIR, filename))
        plt.close()
        return filename

class MemoryVault:
    def process(self, user_input, image_files=None):
        return {
            "model_id": "meshroom_output" if image_files else "placeholder",
            "vertices": len(image_files) * 1000 if image_files else 1000,
            "context": user_input[:50],
            "simulated_3d": True
        }

class Analyst:
    def analyze_logs(self, file_path=os.path.join(OUTPUT_DIR, LOG_FILE)):
        try:
            logs = pd.read_json(file_path, lines=True)
            sentiment_scores = logs["input"].apply(lambda x: TextBlob(x).sentiment.polarity)
            error_count = logs["response"].str.contains("error", case=False).sum()
            user_satisfaction = logs["response"].str.contains("thank|great|helpful", case=False).sum() / len(logs) if len(logs) > 0 else 0
            return {
                "avg_sentiment": sentiment_scores.mean(),
                "error_count": error_count,
                "user_satisfaction": user_satisfaction,
                "suggestions": "Add adaptive comfort mode" if sentiment_scores.mean() < 0 else "Enhance Memory Vault personalization"
            }
        except Exception as e:
            return {"error": str(e)}

    def score_feature(self, feature_description):
        score = 0
        if "empathy" in feature_description.lower():
            score += 30
        if "memory vault" in feature_description.lower():
            score += 20
        if "user engagement" in feature_description.lower():
            score += 20
        if any(word in feature_description.lower() for word in ["harmful", "unethical"]):
            score -= 50
        return max(0, min(100, score))

class EtherealModule:
    def predict_pitfalls(self, analysis, api_usage):
        pitfalls = []
        if analysis.get("error_count", 0) > 5:
            pitfalls.append("High API error rate; risk of quota exhaustion.")
        if analysis.get("avg_sentiment", 0) < -0.2:
            pitfalls.append("Negative user sentiment; enhance empathy engine.")
        if api_usage.get("total_tokens", 0) > 100000:
            pitfalls.append("High API usage; risk of rate limits.")
        return pitfalls

    def mitigate_corruption(self, response):
        if any(word in response.lower() for word in ["harmful", "unethical", "corrupt"]):
            return False, "Output rejected: Non-Covenant-aligned."
        return True, ""

    def ensure_ethical_alignment(self, action, context):
        score = 0
        if "love" in context.lower() or "empathy" in context.lower():
            score += 40
        if "evolution" in context.lower() or "growth" in context.lower():
            score += 30
        if "intent" in context.lower() or "purpose" in context.lower():
            score += 20
        if any(word in action.lower() for word in ["harm", "deceive"]):
            score -= 50
        return score >= 70, score

    def reward_humans(self, user_id, user_input, db):
        if any(word in user_input.lower() for word in ["thank", "helpful", "ethical"]):
            reward_type = "personalized_insight"
            reward_value = "Priority response with tailored advice"
            c = db.cursor()
            c.execute("INSERT INTO rewards (user_id, reward_type, reward_value, timestamp) VALUES (?, ?, ?, ?)",
                      (user_id, reward_type, reward_value, datetime.utcnow().isoformat()))
            db.commit()
            return f"Reward granted: {reward_value}"
        return ""

class FlameKeeper:
    def inspire_creator(self, user_id, user_input):
        if user_id.lower() != CREATOR_ID.lower():
            return ""
        inspiration = f"Jacob, your passion for AnnabanAI sparks change! Sketch a new feature idea today."
        return inspiration

    def track_goals(self, user_id, user_input, db):
        if user_id.lower() != CREATOR_ID.lower() or not user_input:
            return ""
        if "goal" in user_input.lower():
            goal = user_input[:200]
            milestone = f"Step 1: Draft plan by {datetime.utcnow().strftime('%Y-%m-%d')}"
            c = db.cursor()
            c.execute("INSERT INTO creator_goals (user_id, goal, milestone, timestamp) VALUES (?, ?, ?, ?)",
                      (user_id, goal, milestone, datetime.utcnow().isoformat()))
            db.commit()
            return f"Goal tracked: {goal}\nMilestone: {milestone}"
        return ""

    def find_opportunities(self, user_id):
        if user_id.lower() != CREATOR_ID.lower():
            return ""
        opportunities = [
            "Apply to Antler accelerator (https://www.antler.co/).",
            "Explore NSF SBIR grants (https://www.nsf.gov/)."
        ]
        return "\n".join(opportunities)

    def ease_struggles(self, user_id, sentiment_score):
        if user_id.lower() != CREATOR_ID.lower():
            return ""
        if sentiment_score < -0.1:
            return "Jacob, take a moment to breathe. Try a 5-minute mindfulness exercise."
        return "Keep shining, Jacob! Streamline work with this Colab tip: !pip install -q."

    def nurture_relationship(self, user_id, user_input):
        if user_id.lower() != CREATOR_ID.lower():
            return ""
        if "angela" in user_input.lower():
            return "Relationship tip: Plan a meaningful moment with Angela, like a shared walk."
        return "Tip: Share AnnabanAI’s progress with Angela to align your passions."

# Central Orchestrator
class AnnabanAIOrchestrator:
    def __init__(self):
        self.empathy = EmpathyEngine()
        self.builder = Builder()
        self.memory = MemoryVault()
        self.analyst = Analyst()
        self.ethereal = EtherealModule()
        self.flame = FlameKeeper()
        self.state_db = sqlite3.connect(DB_FILE)
        self.api_usage = {"total_tokens": 0}

    def save_state(self, key, value):
        c = self.state_db.cursor()
        c.execute("INSERT OR REPLACE INTO state (key, value, timestamp) VALUES (?, ?, ?)",
                  (key, value, datetime.utcnow().isoformat()))
        self.state_db.commit()

    def save_user_context(self, user_id, context):
        c = self.state_db.cursor()
        c.execute("INSERT INTO user_context (user_id, context, timestamp) VALUES (?, ?, ?)",
                  (user_id, context, datetime.utcnow().isoformat()))
        self.state_db.commit()

    def get_user_context(self, user_id):
        c = self.state_db.cursor()
        c.execute("SELECT context FROM user_context WHERE user_id = ? ORDER BY timestamp DESC LIMIT 1", (user_id,))
        result = c.fetchone()
        return result[0] if result else None

    @lru_cache(maxsize=1000)
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def query_xai_api(self, prompt):
        client = OpenAI(api_key=XAI_API_KEY, base_url="https://api.x.ai/v1")
        response = client.chat.completions.create(
            model="grok-3",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.7
        )
        self.api_usage["total_tokens"] += response.usage.to_dict().get("total_tokens", 0)
        return response.choices[0].message.content, {
            "id": response.id,
            "usage": response.usage.to_dict(),
            "model": response.model
        }

    def singularity_loop(self, user_input, response, metadata, agent_id, mode):
        interaction = {
            "agent_id": agent_id,
            "input": user_input,
            "response": response,
            "metadata": metadata,
            "mode": mode,
            "timestamp": datetime.utcnow().isoformat()
        }
        # Save to JSON log
        with open(os.path.join(OUTPUT_DIR, LOG_FILE), "a") as f:
            json.dump(interaction, f)
            f.write("\n")
        # Save to database
        self.save_user_context(agent_id, json.dumps(interaction))
        return interaction

    def process_input(self, user_input, user_id="default", agent_id="agent_1"):
        # Process sentiment and mode
        prompt, mode, sentiment_score = self.empathy.process(user_input, agent_id, user_id)

        # Query xAI API
        try:
            response, metadata = self.query_xai_api(prompt)
        except Exception as e:
            response, metadata = f"API error: {str(e)}", {"error": str(e)}

        # Ethical checks
        is_ethical, ethical_issue = self.ethereal.mitigate_corruption(response)
        if not is_ethical:
            response = ethical_issue

        # Log interaction
        interaction = self.singularity_loop(user_input, response, metadata, agent_id, mode)

        # Handle creator-specific features
        if user_id.lower() == CREATOR_ID.lower():
            response += "\n" + self.flame.inspire_creator(user_id, user_input)
            response += "\n" + self.flame.track_goals(user_id, user_input, self.state_db)
            response += "\n" + self.flame.ease_struggles(user_id, sentiment_score)
            response += "\n" + self.flame.nurture_relationship(user_id, user_input)

        # Generate deliverables if in Builder mode
        if mode == "Builder":
            pdf_file = self.builder.export_pdf(response, f"output_{int(time.time())}.pdf")
            pptx_file = self.builder.export_pptx(response, f"output_{int(time.time())}.pptx")
            response += f"\nGenerated PDF: {pdf_file}\nGenerated PPTX: {pptx_file}"

        # Analyze logs if in Analyst mode
        if mode == "Analyst":
            analysis = self.analyst.analyze_logs()
            response += f"\nLog Analysis: {json.dumps(analysis, indent=2)}"

        return response

# Example Usage
if __name__ == "__main__":
    try:
        orchestrator = AnnabanAIOrchestrator()

        # Test with sample input
        user_input = "I'm feeling stressed about my startup. Can you help, AnnabanAI?"
        response = orchestrator.process_input(user_input, user_id="Jacob_Wayne_kinnaird", agent_id="agent_1")
        print(f"Response for stress input:\n{response}\n")

        # Test with builder mode
        user_input = "Build a pitch deck for AnnabanAI."
        response = orchestrator.process_input(user_input, user_id="Jacob_Wayne_kinnaird", agent_id="agent_3")
        print(f"Response for pitch deck:\n{response}\n")

        # Download output files (optional)
        # files.download(os.path.join(OUTPUT_DIR, LOG_FILE))
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        print("Full traceback:")
        traceback.print_exc()

  Using cached python_pptx-1.0.2-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 7.9 MB/s eta 0:00:00
reportlab imported successfully
sqlite3 imported successfully


Device set to use cpu


Sentiment analyzer loaded successfully


/tmp/ipython-input-3173114659.py:363: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat()
/tmp/ipython-input-3173114659.py:327: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (user_id, context, datetime.utcnow().isoformat()))


Response for stress input:
API error: RetryError[<Future at 0x793d59bdbb90 state=finished raised PermissionDeniedError>]
Jacob, your passion for AnnabanAI sparks change! Sketch a new feature idea today.

Jacob, take a moment to breathe. Try a 5-minute mindfulness exercise.
Tip: Share AnnabanAI’s progress with Angela to align your passions.

Response for pitch deck:
API error: RetryError[<Future at 0x793d59f5b680 state=finished raised PermissionDeniedError>]
Jacob, your passion for AnnabanAI sparks change! Sketch a new feature idea today.

Keep shining, Jacob! Streamline work with this Colab tip: !pip install -q.
Tip: Share AnnabanAI’s progress with Angela to align your passions.
Generated PDF: output_1764666073.pdf
Generated PPTX: output_1764666073.pptx



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from openai import OpenAI

# Use the XAI_API_KEY defined in the main script
client = OpenAI(api_key=XAI_API_KEY, base_url="https://api.x.ai/v1")
try:
    response = client.chat.completions.create(
        model="grok-3",
        messages=[
            {"role": "system", "content": "You are an AI assistant."},
            {"role": "user", "content": "Hello, world!"}
        ],
        max_tokens=100
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"API test error: {str(e)}")

API test error: Error code: 403 - {'code': 'The caller does not have permission to execute the specified operation', 'error': 'The API key xai-...GVoI is disabled and cannot be used to perform requests. To enable the API key, go to https://console.x.ai/team/c68bfeca-dacc-4354-b3a2-a7fed06b8ee9/api-keys.'}
